In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importing all the libraries that we would be using**

In [0]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [0]:
##import loan.csv file
d=pd.read_csv('/content/drive/Shared drives/OPIM 5512 - Python Project/loan.csv', low_memory=False)

In [0]:
df = d.copy(deep=True)

**Working on a copy of the dataframe so that I do not have to re-read the entire dataset again in order to save memory**


**Checking if the dataframe has loaded properly**

In [0]:
df = df.filter(['acc_now_delinq',
  'addr_state',
  'annual_inc',
  'annual_inc_joint',
  'application_type',
  'dti',
  'emp_length',
  'fico_range_high',
  'fico_range_low',
  'home_ownership',
  'int_rate',
  'last_fico_range_high',
  'last_fico_range_low',
  'loan_amnt',
  'open_acc',
  'pub_rec',
  'purpose'])
df.head()

,acc_now_delinq,addr_state,annual_inc,annual_inc_joint,application_type,dti,emp_length,home_ownership,int_rate,loan_amnt,open_acc,pub_rec,purpose
0,0.0,AZ,24000.0,NaN,INDIVIDUAL,27.65,10+ years,RENT,10.65,5000.0,3.0,0.0,credit_card
1,0.0,GA,30000.0,NaN,INDIVIDUAL,1.00,< 1 year,RENT,15.27,2500.0,3.0,0.0,car
2,0.0,IL,12252.0,NaN,INDIVIDUAL,8.72,10+ years,RENT,15.96,2400.0,2.0,0.0,small_business
3,0.0,CA,49200.0,NaN,INDIVIDUAL,20.00,10+ years,RENT,13.49,10000.0,10.0,0.0,other
4,0.0,OR,80000.0,NaN,INDIVIDUAL,17.94,1 year,RENT,12.69,3000.0,15.0,0.0,other


**Printing the basic statistical details of the dataset**

In [0]:
df.shape

(887379, 13)

**Checking the dimensions**

**Finding the the count and percentage of values that are missing in the dataframe and printing the columns with null count more than 0 in descending order**

In [0]:
df_null = pd.DataFrame({'Count': df.isnull().sum(), 'Percent': 100*df.isnull().sum()/len(df)})

df_null[df_null['Count'] > 0].sort_values(by=['Percent'], ascending=False)

,Count,Percent
annual_inc_joint,886868,99.942415
emp_length,44825,5.051393
acc_now_delinq,29,0.003268
open_acc,29,0.003268
pub_rec,29,0.003268
annual_inc,4,0.000451


In [0]:
missing_data = [i for i in df.count()<887379 *0.30]
df.drop(df.columns[missing_data],axis=1,inplace=True)
df.shape

(887379, 12)

In [0]:
df['purpose_g'] = np.nan

df.loc[(df['purpose'] == 'debt_consolidation')|(df['purpose'] =="credit_card"), 'purpose_g'] = 'debt' 
df.loc[(df['purpose'] == 'home_improvement')|(df['purpose'] =="major_purchase")|
                 (df['purpose'] == 'car')|(df['purpose'] =="house")|
                 (df['purpose'] == 'vacation')|(df['purpose'] =="renewable_energy"),
                 'purpose_g'] = 'personal_major' 
df.loc[(df['purpose'] == 'small_business')|(df['purpose'] =="medical")|
                 (df['purpose'] == 'moving')|(df['purpose'] =="wedding")|
                 (df['purpose'] == 'educational'),
                 'purpose_g'] = 'personal_minor'
df.loc[(df['purpose'] == 'other'), 'purpose_g'] = 'other'
df.drop(['purpose'], axis=1, inplace = True)

In [0]:
df.purpose_g

0                   debt
1         personal_major
2         personal_minor
3                  other
4                  other
               ...      
887374              debt
887375    personal_major
887376              debt
887377              debt
887378              debt
Name: purpose_g, Length: 887379, dtype: object

In [0]:
# create region of residence based on state
west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 
              'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 
            'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']

def finding_regions(state):
    if state in west:
        return 'West'
    elif state in south_west:
        return 'SouthWest'
    elif state in south_east:
        return 'SouthEast'
    elif state in mid_west:
        return 'MidWest'
    elif state in north_east:
        return 'NorthEast'
    
df['region'] = df['addr_state'].apply(finding_regions)

# drop rows without region value
df.dropna(subset=['region'],inplace=True)
df.drop(['addr_state'], axis=1, inplace = True)

In [0]:
df['emp_length'].fillna(value=0,inplace=True)

df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)

In [0]:
df.shape

(887379, 12)

In [0]:
df_null = pd.DataFrame({'Count': df.isnull().sum(), 'Percent': 100*df.isnull().sum()/len(df)})

df_null[df_null['Count'] > 0].sort_values(by=['Percent'], ascending=False)

,Count,Percent
acc_now_delinq,29,0.003268
open_acc,29,0.003268
pub_rec,29,0.003268
annual_inc,4,0.000451


In [0]:
df.dropna(axis=0, inplace=True)
df.shape

(887350, 12)

In [0]:
df_null = pd.DataFrame({'Count': df.isnull().sum(), 'Percent': 100*df.isnull().sum()/len(df)})

df_null[df_null['Count'] > 0].sort_values(by=['Percent'], ascending=False)

,Count,Percent


In [0]:
from sklearn import preprocessing
count = 0
for col in df:
    if df[col].dtype == 'object':
        if len(list(df[col].unique())) <= 2:     
            le = preprocessing.LabelEncoder()
            df[col] = le.fit_transform(df[col])
            count += 1
            print (col)
            
print('%d columns were label encoded.' % count)

application_type
1 columns were label encoded.


In [0]:
df = pd.get_dummies(df)
print(df.shape)

(887350, 34)


In [0]:
df.head()

,acc_now_delinq,annual_inc,application_type,dti,int_rate,loan_amnt,open_acc,pub_rec,emp_length_0,emp_length_1,emp_length_10,emp_length_2,emp_length_3,emp_length_4,emp_length_5,emp_length_6,emp_length_7,emp_length_8,emp_length_9,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_g_debt,purpose_g_other,purpose_g_personal_major,purpose_g_personal_minor,region_MidWest,region_NorthEast,region_SouthEast,region_SouthWest,region_West
0,0.0,24000.0,0,27.65,10.65,5000.0,3.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0
1,0.0,30000.0,0,1.00,15.27,2500.0,3.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
2,0.0,12252.0,0,8.72,15.96,2400.0,2.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0
3,0.0,49200.0,0,20.00,13.49,10000.0,10.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
4,0.0,80000.0,0,17.94,12.69,3000.0,15.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1


In [0]:
df_new = df.copy()

In [0]:
df_new.drop('int_rate',axis=1,inplace=True)

In [0]:
df_new.head()

,acc_now_delinq,annual_inc,application_type,dti,loan_amnt,open_acc,pub_rec,emp_length_0,emp_length_1,emp_length_10,emp_length_2,emp_length_3,emp_length_4,emp_length_5,emp_length_6,emp_length_7,emp_length_8,emp_length_9,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_g_debt,purpose_g_other,purpose_g_personal_major,purpose_g_personal_minor,region_MidWest,region_NorthEast,region_SouthEast,region_SouthWest,region_West
0,0.0,24000.0,0,27.65,5000.0,3.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0
1,0.0,30000.0,0,1.00,2500.0,3.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
2,0.0,12252.0,0,8.72,2400.0,2.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0
3,0.0,49200.0,0,20.00,10000.0,10.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
4,0.0,80000.0,0,17.94,3000.0,15.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1


In [0]:
df_int = df['int_rate']

In [0]:
df_int

0         10.65
1         15.27
2         15.96
3         13.49
4         12.69
          ...  
887374    11.99
887375    11.99
887376    15.99
887377    19.99
887378    11.99
Name: int_rate, Length: 887350, dtype: float64

In [0]:
df_new.insert(0, 'int_rate', df_int)

In [0]:
df_new.head()

,int_rate,acc_now_delinq,annual_inc,application_type,dti,loan_amnt,open_acc,pub_rec,emp_length_0,emp_length_1,emp_length_10,emp_length_2,emp_length_3,emp_length_4,emp_length_5,emp_length_6,emp_length_7,emp_length_8,emp_length_9,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_g_debt,purpose_g_other,purpose_g_personal_major,purpose_g_personal_minor,region_MidWest,region_NorthEast,region_SouthEast,region_SouthWest,region_West
0,10.65,0.0,24000.0,0,27.65,5000.0,3.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0
1,15.27,0.0,30000.0,0,1.00,2500.0,3.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
2,15.96,0.0,12252.0,0,8.72,2400.0,2.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0
3,13.49,0.0,49200.0,0,20.00,10000.0,10.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
4,12.69,0.0,80000.0,0,17.94,3000.0,15.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1


**Linear Regression - OLS**

In [0]:
import statsmodels.api as sm

In [0]:
X, y = df_new.iloc[:, 1:].values, df_new.iloc[:, 0].values

In [0]:
model=sm.OLS(y,X)

In [0]:
results=model.fit()

In [0]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     2692.
Date:                Mon, 06 Apr 2020   Prob (F-statistic):               0.00
Time:                        20:18:20   Log-Likelihood:            -2.5327e+06
No. Observations:              887350   AIC:                         5.065e+06
Df Residuals:                  887320   BIC:                         5.066e+06
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.6209      0.057     28.202      0.000       1.508       1.734
x2           -8.7e-06   7.47e-08   -116.528      0.000   -8.85e-06   -8.55e-06
x3             0.3303      0.186      1.772      0.076      -0.035       0.696
x4             0.0191      0.000     72.062      0.000       0.019       0.020
x5             0.0001   5.88e-07    211.712      0.000       0.000       0.000
x6            -0.0216      0.001    -24.621      0.000      -0.023      -0.020
x7             0.5102      0.008     66.109      0.000       0.495       0.525
x8             1.7240      0.057     30.112      0.000       1.612       1.836
x9             1.5741      0.055     28.455      0.000       1.466       1.682
x10            1.7070      0.055     31.176      0.000       1.600       1.814
x11            1.6241      0.056     29.020      0.000       1.514       1.734
x12            1.6160      0.056     28.766      0.000       1.506       1.726
x13            1.6553      0.057     29.141      0.000       1.544       1.767
x14            1.7184      0.057     30.351      0.000       1.607       1.829
x15            1.8678      0.057     32.573      0.000       1.755       1.980
x16            1.9115      0.057     33.418      0.000       1.799       2.024
x17            1.7186      0.057     29.993      0.000       1.606       1.831
x18            1.7429      0.058     30.012      0.000       1.629       1.857
x19            2.2531      2.118      1.064      0.288      -1.899       6.405
x20            2.5101      0.322      7.807      0.000       1.880       3.140
x21            4.8114      0.624      7.712      0.000       3.589       6.034
x22            3.1357      0.419      7.483      0.000       2.314       3.957
x23            2.8351      0.322      8.814      0.000       2.205       3.466
x24            3.3142      0.321     10.309      0.000       2.684       3.944
x25            3.2698      0.149     21.970      0.000       2.978       3.562
x26            5.7795      0.150     38.637      0.000       5.486       6.073
x27            3.7879      0.149     25.389      0.000       3.495       4.080
x28            6.0224      0.150     40.102      0.000       5.728       6.317
x29            3.7997      0.119     31.850      0.000       3.566       4.034
x30            3.6970      0.119     31.009      0.000       3.463       3.931
x31            3.9265      0.119     32.940      0.000       3.693       4.160
x32            3.7519      0.119     31.419      0.000       3.518       3.986
x33            3.6845      0.119     30.911      0.000       3.451       3.918
==============================================================================
Omnibus:                   167844.118   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4091165.250
Sk

**Linear Regression - sklearn**

In [0]:
from sklearn.model_selection import train_test_split
X1, y1 = df_new.iloc[:, 1:].values, df_new.iloc[:, 0].values
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

In [0]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
model2 = regressor.fit(X1_train,y1_train)

In [0]:
feature_col = df_new.columns.values[1:]
feature = {'indicators':feature_col,
       'coef':model2.coef_}
results2 = pd.DataFrame(feature)

In [0]:
results2.sort_values('coef',ascending=False,inplace=True)
results2

,indicators,coef
20,home_ownership_NONE,1.814440
0,acc_now_delinq,1.614415
27,purpose_g_personal_minor,1.292117
25,purpose_g_other,1.074897
6,pub_rec,0.504724
2,application_type,0.294257
15,emp_length_7,0.205724
14,emp_length_6,0.159449
30,region_SouthEast,0.158428
23,home_ownership_RENT,0.134803
